# Instalar zigbee2mqtt

Con [zigbee2mqtt](https://www.zigbee2mqtt.io/) podemos tener una red zigbee y usarla en home assistant mediante el protocolo mqtt

## Controlador o dongle

En primer lugar necesitamos tener un controlador que se conectará a la raspberry, en mi caso tengo el `SONOFF Zigbee 3.0 USB Dongle`

## Instalación de zigbee2mqtt con docker

En la página de [instalación con docker](https://www.zigbee2mqtt.io/guide/installation/02_docker.html) están las instrucciones

### Actualización de paquetes para raspberry

En versiones anteriores de Debian/Raspberry Pi OS era necesario instalar paquetes adicionales, pero en las versiones actuales (Debian Bookworm/Raspberry Pi OS 12) estos paquetes ya vienen incluidos por defecto.

Si estás usando una versión anterior, puedes verificar si tienes libseccomp2 instalado:

``` bash
dpkg -l | grep libseccomp2
```

Si no está instalado:
``` bash
sudo apt update
sudo apt install -y libseccomp2
```

### Carpetas para el contenedor

Ahora nos creamos las carpetas necesarias para el contenedor

``` bash
mkdir ~/docker/zigbee2mqtt
mkdir ~/docker/zigbee2mqtt/data
```

### Encontrar la ruta del controlador

Para encontrar la ruta del controlador ejecutar

``` bash
ls -l /dev/serial/by-path/
```

En mi caso aparece `lrwxrwxrwx 1 root root 13 Aug 23 05:46 platform-fd500000.pcie-pci-0000:01:00.0-usb-0:1.1:1.0-port0 -> ../../ttyUSB0`, por lo que sé que su path es `/dev/ttyUSB0`

### Crear archivo de configuración

Crear el archivo de configuración

``` bash
touch ~/docker/zigbee2mqtt/data/configuration.yaml
```

Lo abrimos con nano

``` bash
nano ~/docker/zigbee2mqtt/data/configuration.yaml
```

Y escribimos lo siguiente

``` bash
# Home Assistant integration (MQTT discovery)
homeassistant: true

# allow new devices to join
permit_join: true

# MQTT settings
mqtt:
  # MQTT server URL
  server: mqtt://localhost:1883
  # MQTT base topic for zigbee2mqtt MQTT messages
  base_topic: zigbee2mqtt
  # MQTT server authentication, uncomment if required:
  user: <USER>
  password: <PASSWORD>

# Serial settings
serial:
  # Location of CC2531 USB sniffer
  port: /dev/ttyACM0

frontend: true

advanced:
  homeassistant_legacy_entity_attributes: false
  legacy_api: false
  legacy_availability_payload: false
  network_key: GENERATE

device_options:
  legacy: false

devices:
  '0x00124b0024c8a0fe':
    friendly_name: '0x00124b0024c8a0fe'
```

Poner en `<USER>` y `<PASSWORD>` el nombre y usuario que elegimos cuando instalamos el contenedor de mqtt e hicimos `mosquitto_passwd -c passwordfile.txt <USER>`
```

### docker compose

Ir a portainer y en el stack de domotica añadir

``` bash
  zigbee2mqtt:
    container_name: zigbee2mqtt
    image: koenkk/zigbee2mqtt:1.33.2
    restart: unless-stopped
    volumes:
      - /home/raspihome/docker/zigbee2mqtt/data:/app/data
      - /run/udev:/run/udev:ro
    ports:
      - 8080:8080
    environment:
      - TZ=Europe/Madrid
    devices:
      - /dev/ttyUSB0:/dev/ttyACM0
    network_mode: host
    depends_on:
      - mqtt
```

Y darle a deploy

Una vez levantado el contenedor podemos ir a `http://<IP>:8080/#/` para ver la red zigbee